In [1]:
## Importing the required python libraries 
import os, sys
import warnings
import timeit
import numpy as np
import pandas as pd 
import pkg_resources
import matplotlib.pyplot as plt
import re
import seaborn as sns
from collections import defaultdict


In [2]:
## Importing pergene files 

pergene_files=[]
#data_dir= "../satay/data_files/data_unmerged/"
#data_dir="../transposonmapper/data_files/files4test/"
data_dir="../postprocessed-data/"
#data_dir="../transposonmapper/data_files/"
for root, dirs, files in os.walk(data_dir):
    for file in files:
        if file.endswith("pergene_insertions.xlsx"):
            pergene_files.append(os.path.join(root, file))

list_data=[]
for i in pergene_files:
    list_data.append(pd.read_excel(i,engine='openpyxl',index_col="Unnamed: 0"))

keys=[]
for i in np.arange(0,len(pergene_files)):
    keys.append(pergene_files[i].split("/")[-1].split("_")[0]+"_"+pergene_files[i].split("/")[-1].split("_")[1])

list_data_pd=pd.concat(list_data,axis=0,keys=keys)

In [15]:
## Importing postproccesed data from pipeline

data_post=[]

for i in keys:
    data_post.append(pd.read_excel(data_dir+i+".xlsx",index_col="Unnamed: 0",engine='openpyxl'))
    
list_data_post=pd.concat(data_post,axis=0,keys=keys)
list_data_post.drop(columns=["Feature_alias","Feature_name"],inplace=True)
list_data_post.fillna(1,inplace=True)


In [19]:
list_data_post.head()
# list_data_pd.head()

Standard_name   Feature_type  Essentiality Chromosome    Position  \
dbem3_b 0     noncoding              1           1.0          I      [0, 1]   
        1        TEL01L       Telomere           1.0          I    [2, 335]   
        2       YAL069W  Gene; Dubious           0.0          I  [336, 650]   
        3        TEL01L       Telomere           1.0          I  [651, 707]   
        4        ARS102            ARS           1.0          I  [708, 776]   

           Nbasepairs  Ninsertions  Ninsertions_truncatedgene  Nreads  \
dbem3_b 0           2            0                          0       0   
        1         334           38                         38    2356   
        2         315           51                         20    2832   
        3          57            2                          2      28   
        4          69            6                          6      11   

                                                 Nreads_list  \
dbem3_b 0                                               [0.]   
        1  [  0.   0.   0.   0.   0.   0.   0.   0.   0. ...   
        2  [  0.   0.   0. 327.   0.   0.   0.   0.   0. ...   
        3  [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  ...   
        4  [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. ...   

           Nreads_truncatedgene  Nreadsperinsrt  Nreadsperinsrt_truncatedgene  
dbem3_b 0                     0        0.000000                      0.000000  
        1                  2356       63.675676                     62.000000  
        2                  1406       56.640000                     70.300000  
        3                    28        0.000000                      0.000000  
        4                    11        2.200000                      1.833333

In [73]:
## Importing gene protein length , and protein domains coordinates of every gene 

protein_length=pd.read_csv('../data/Gene_Protein_length_yeastmine.tsv',sep="\t")
protein_length.index=protein_length.loc[:,"Gene Name"]
protein_length.drop(columns="Gene Name",inplace=True)

domains_coordinates=pd.read_csv('../data/Domains_all_genes_protein_coordinates_yeastmine.tsv',sep="\t")

domains_coordinates.dropna(inplace=True)
gene_with_introns=pd.read_csv('../data/Gene_Introns.tsv',sep="\t")



In [77]:
## Combine protein length and protein domains coordinates
domains_coordinates.loc[:,"Protein Length"]=np.nan
for gene_name in protein_length.index: 
    tmp=np.where(domains_coordinates.loc[:,"Gene Name"]==gene_name)[0]
    if tmp.size!=0: 
        for i in np.arange(0,len(tmp)):
            domains_coordinates.loc[tmp[i], "Protein Length"]=protein_length.loc[gene_name, "Protein Length"]

domains_coordinates.dropna(inplace=True) # to drop the proteins that do not have protein domains annotated

In [78]:
domains_coordinates.head() 

,Gene Name,Protein Domain,Protein domain description,Protein start location,Protein End location,Method,Domain ID,Protein Length
0,AAC1,PF00153,Mitochondrial carrier protein; Mitochondrial s...,10.0,106.0,Pfam,2106639.0,309.0
1,AAC1,PF00153,Mitochondrial carrier protein; Mitochondrial s...,116.0,210.0,Pfam,2106640.0,309.0
2,AAC1,PF00153,Mitochondrial carrier protein; Mitochondrial s...,217.0,304.0,Pfam,2106641.0,309.0
3,AAC3,PF00153,Mitochondrial carrier protein; Mitochondrial s...,9.0,105.0,Pfam,2110562.0,307.0
4,AAC3,PF00153,Mitochondrial carrier protein; Mitochondrial s...,115.0,208.0,Pfam,2110563.0,307.0
